# 1  生成可交互式地图

In [ ]:
# import 相关库
import geemap
import os
import ee

# 如果后续GEE授权认证时出现问题，则需设置代理地址，并根据本机地址进行调整
# 如果没有问题，可以跳过下段代码
os.environ['HTTP_PROXY'] = 'http://127.0.0.1:10809'
os.environ['HTTPS_PROXY'] = 'http://127.0.0.1:10809'

In [ ]:
import json
import requests
from geemap import geojson_to_ee, ee_to_geojson
from ipyleaflet import GeoJSON, Marker, MarkerCluster

In [ ]:
# 更新geemap
geemap.update_package()

## 1.1  生成基础地图

In [ ]:
Map = geemap.Map()
# Defalut: Map.setCenter(40, -100, 4)
Map

## 1.2  更改/添加底层地图：Map.add_basemap

In [ ]:
Map.add_basemap('HYBRID')

In [ ]:
Map.basemap_demo()

# 2  交互式工具

## 2.1  inspector

In [ ]:
# 初始化
Map = geemap.Map()

# 导入GEE数据
dem = ee.Image('USGS/SRTMGL1_003')  # SRTM高程数据
landcover = ee.Image("ESA/GLOBCOVER_L4_200901_200912_V2_3").select('landcover')
states = ee.FeatureCollection("TIGER/2018/States")

# 设置显示参数
vis_params = {
    'min': 0,
    'max': 4000,
    'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']
}

# 利用addLayer添加图层
Map.addLayer(dem, vis_params, 'STRM DEM', True, 0.5)
Map.addLayer(landcover, {}, 'Land cover')
Map.addLayer(states, {}, "US States")

Map

## 2.2  draw features

In [ ]:
# 获取添加的特征列表
Map.draw_features

In [ ]:
# 获取添加的最后一个特征
Map.draw_last_feature

In [ ]:
# 将所有特征合并到一个集合中，获取其在states中的边界
roi = ee.FeatureCollection(Map.draw_features)
selected_states = states.filterBounds(roi)
Map.addLayer(selected_states, {}, "Selected states")

In [ ]:
# 裁剪所需要的区域
clipped_dem = dem.clip(selected_states)
Map.addLayer(clipped_dem, vis_params, 'Clipped image')

## 2.3  split_map

In [ ]:
# https://developers.google.com/earth-engine/datasets/catalog/USGS_NLCD
# 定义一个函数，返回GEE数据集包含的所有子集ID
def Get_GEEid(para):
    collection = ee.ImageCollection(para)
    return collection.aggregate_array('system:id').getInfo()


USGS_ID = Get_GEEid("USGS/NLCD")
USGS_ID

In [ ]:
# 分别选取2001年和2016年的土地利用数据进行对比
nlcd_old = ee.Image(USGS_ID[1]).select('landcover')
nlcd_new = ee.Image(USGS_ID[-1]).select('landcover')

# 把GEE的图层转换为ipyleaflet TileLayer
left_layer = geemap.ee_tile_layer(nlcd_old, {}, 'NLCD old')
right_layer = geemap.ee_tile_layer(nlcd_new, {}, 'NLCD new')

Map = geemap.Map()
Map.split_map(left_layer, right_layer)
Map

## 2.4  Plotting

In [ ]:
Map = geemap.Map()

landsat7 = ee.Image('LE7_TOA_5YEAR/1999_2003') \
    .select([0, 1, 2, 3, 4, 6])
landsat_vis = {'bands': ['B4', 'B3', 'B2'], 'gamma': 1.4}
Map.addLayer(landsat7, landsat_vis, "LE7_TOA_5YEAR/1999_2003")

GPM = ee.ImageCollection('NASA/GPM_L3/IMERG_V06') \
    .filter(ee.Filter.date('2016-01-01', '2016-06-01'))
GPM_vis = {
    'min': 0.0,
    'max': 1000.0,
    'gamma': 2.5,
}
Map.addLayer(GPM, GPM_vis, 'NASA/GPM_L3/IMERG_V06')

Map

In [ ]:
在图层右上角勾选【Use Plotting】
在图层中点击任意一点即可查看信息

## 2.5  legend

In [ ]:
# 使用builtin_lengends添加以下遥感图像的图例
legends = geemap.builtin_legends
for legend in legends:
    print(legend)

In [ ]:
Map = geemap.Map()
Map.add_basemap('HYBRID')
landcover = ee.Image('USGS/NLCD/NLCD2016').select('landcover')
Map.addLayer(landcover, {}, 'NLCD Land Cover')
Map.add_legend(builtin_legend='NLCD')
Map

In [ ]:
# 自定义图例
Map = geemap.Map()

#（1）通过定义键和颜色
legend_keys = ['One', 'Two', 'Three', 'Four', 'ect']
 #colorS can be defined using either hex code or RGB (0-255, 0-255, 0-255)
legend_colors = ['#8DD3C7', '#FFFFB3', '#BEBADA', '#FB8072', '#80B1D3']
# legend_colors = [(255, 0, 0), (127, 255, 0), (127, 18, 25), (36, 70, 180), (96, 68 123)]
Map.add_legend(legend_keys=legend_keys, legend_colors=legend_colors, position='bottomleft')

#（2）通过定义字典
legend_dict = {
    '11 Open Water': '466b9f',
    '12 Perennial Ice/Snow': 'd1def8',
    '21 Developed, Open Space': 'dec5c5',
    '22 Developed, Low Intensity': 'd99282',
    '23 Developed, Medium Intensity': 'eb0000',
    '24 Developed High Intensity': 'ab0000',
    '31 Barren Land (Rock/Sand/Clay)': 'b3ac9f',
    '41 Deciduous Forest': '68ab5f',
    '42 Evergreen Forest': '1c5f2c',
    '43 Mixed Forest': 'b5c58f',
    '51 Dwarf Scrub': 'af963c',
    '52 Shrub/Scrub': 'ccb879',
    '71 Grassland/Herbaceous': 'dfdfc2',
    '72 Sedge/Herbaceous': 'd1d182',
    '73 Lichens': 'a3cc51',
    '74 Moss': '82ba9e',
    '81 Pasture/Hay': 'dcd939',
    '82 Cultivated Crops': 'ab6c28',
    '90 Woody Wetlands': 'b8d9eb',
    '95 Emergent Herbaceous Wetlands': '6c9fb8'
}
landcover = ee.Image('USGS/NLCD/NLCD2016').select('landcover')
Map.addLayer(landcover, {}, 'NLCD Land Cover')
Map.add_legend(legend_title="NLCD Land Cover Classification", legend_dict=legend_dict, position='bottomright')

# (3) 通过复制GEE的图例属性
ee_class_table = """

Value	Color	Description
0	1c0dff	Water
1	05450a	Evergreen needleleaf forest
2	086a10	Evergreen broadleaf forest
3	54a708	Deciduous needleleaf forest
4	78d203	Deciduous broadleaf forest
5	009900	Mixed forest
6	c6b044	Closed shrublands
7	dcd159	Open shrublands
8	dade48	Woody savannas
9	fbff13	Savannas
10	b6ff05	Grasslands
11	27ff87	Permanent wetlands
12	c24f44	Croplands
13	a5a5a5	Urban and built-up
14	ff6d4c	Cropland/natural vegetation mosaic
15	69fff8	Snow and ice
16	f9ffa4	Barren or sparsely vegetated
254	ffffff	Unclassified

"""

landcover = ee.Image('MODIS/051/MCD12Q1/2013_01_01') \
    .select('Land_Cover_Type_1')
Map.addLayer(landcover, {}, 'MODIS Land Cover')

legend_dict = geemap.legend_from_ee(ee_class_table)
Map.add_legend(legend_title="MODIS Global Land Cover", legend_dict=legend_dict, position='upleft')

Map

## 2.6  colorbar

In [ ]:
# 定义一个colorbar
width = 250
height = 30
palette = ['blue', 'purple', 'cyan', 'green', 'yellow', 'red']
labels = [-40, 35]
colorbar = geemap.create_colorbar(width=width,
                                  height=height,
                                  palette=palette,
                                  vertical=False,
                                  add_labels=True,
                                  font_size=20,
                                  labels=labels)
geemap.show_image(colorbar)

## 2.7  search

In [ ]:
Map = geemap.Map()
Map

# 在图层左上角点击【search location/data】可以直接获取相关信息

In [ ]:
print(Map.search_locations)
print(Map.search_loc_geom)
print(Map.search_loc_geom.getInfo())

# 3  文件类型

## 3.1  json类型

In [ ]:
file_path = os.path.abspath('../data/us-states.json')

if not os.path.exists(file_path):
    url = 'https://github.com/giswqs/geemap/raw/master/examples/data/us-states.json'
    r = requests.get(url)
    with open(file_path, 'w') as f:
        f.write(r.content.decode("utf-8"))        

with open(file_path) as f:
    json_states = json.load(f)

json_layer = GeoJSON(data=json_states, name='US States JSON', hover_style={'fillColor': 'red' , 'fillOpacity': 0.5})
Map.add_layer(json_layer)

# 转换为GEE类型
ee_states = geojson_to_ee(json_states)
Map.addLayer(ee_states, {}, "US States EE")

# 也可以重新转换为geo类型
# json_states_2 = ee_to_geojson(ee_states)
# json_layer_2 = GeoJSON(data=json_states_2, name='US States EE JSON', hover_style={'fillColor': 'red' , 'fillOpacity': 0.5})
# Map.add_layer(json_layer_2)

## 3.2  shapefiles

In [ ]:
Map = geemap.Map()
# 加载shp文件
states_shp = '../data/us-states.shp'
states = geemap.shp_to_ee(states_shp)
Map.addLayer(states, {}, 'US States')
Map

In [ ]:
# 导出shp文件
geemap.ee_to_shp(states, filename='../data/countries_new.shp')
# 导出其他格式的文件（更改后缀名即可：csv\json\kml\kmz）
geemap.ee_export_vector(states, filename='../data/states.csv')

## 3.3  Image

In [ ]:
# 加载Image图像
image = ee.Image('LE7_TOA_5YEAR/1999_2003')
# 定义下载区域
roi = ee.Geometry.Polygon(
  [[[101.0, 35.0],
    [101.0, 34.0],
    [100.0, 34.0],
    [100.0, 35.0]]], None, False)
# 输出文件路径
out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
filename = os.path.join(out_dir, 'landsat.tif')
filename

In [ ]:
# 裁剪影像
image = image.clip(roi).unmask()
# 输出影像（最后一个选项为是否输出每个波段为一张影像）
geemap.ee_export_image(image, filename=filename, scale=90, region=roi, file_per_band=False)

## 3.4  ImageCollection

In [ ]:
loc = ee.Geometry.Point(-99.2222, 46.7816)
collection = ee.ImageCollection('USDA/NAIP/DOQQ') \
    .filterBounds(loc) \
    .filterDate('2008-01-01', '2020-01-01') \
    .filter(ee.Filter.listContains("system:band_names", "N"))
out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
print(collection.aggregate_array('system:index').getInfo())
geemap.ee_export_image_collection(collection, out_dir=out_dir)

## 3.5  numpy

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

img = ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_038029_20180810') \
  .select(['B4', 'B5', 'B6'])

aoi = ee.Geometry.Polygon(
  [[[-110.8, 44.7],
    [-110.8, 44.6],
    [-110.6, 44.6],
    [-110.6, 44.7]]], None, False)

rgb_img = geemap.ee_to_numpy(img, region=aoi)
print(rgb_img.shape)

In [ ]:
# Scale the data to [0, 255] to show as an RGB image. 
# Adapted from https://bit.ly/2XlmQY8. Credits to Justin Braaten
rgb_img_test = (255*((rgb_img[:, :, 0:3] - 100)/3500)).astype('uint8')
plt.imshow(rgb_img_test)
plt.show()

## 3.6  Gif

In [ ]:
# 加载gif动图
in_gif = os.path.abspath('../data/animation.gif')
geemap.show_image(in_gif)

# 设置输出路径
out_path = r'E:/GEE'
out_gif = os.path.join(out_path, 'Downloads/output.gif')

In [ ]:
geemap.add_text_to_gif(in_gif,
                       out_gif,
                       xy=('5%', '5%'),
                       text_sequence=1984,
                       font_size=30,
                       font_color='#0000ff',
                       duration=100)
geemap.add_text_to_gif(out_gif,
                       out_gif,
                       xy=('30%', '85%'),
                       text_sequence="Las Vegas",
                       font_type="timesbd.ttf",
                       font_color='black')
geemap.show_image(out_gif)

In [ ]:
aoi = ee.Geometry.Polygon(
    [[[-179.0, 78.0], [-179.0, -58.0], [179.0, -58.0], [179.0, 78.0]]], None,
    False)
tempCol = ee.ImageCollection('NOAA/GFS0P25') \
    .filterDate('2018-12-22', '2018-12-23') \
    .limit(24) \
    .select('temperature_2m_above_ground')
videoArgs = {
    'dimensions': 768,
    'region': aoi,
    'framesPerSecond': 10,
    'crs': 'EPSG:3857',
    'min': -40.0,
    'max': 35.0,
    'palette': ['blue', 'purple', 'cyan', 'green', 'yellow', 'red']
}

# 下载Gif
saved_gif = os.path.join(out_path, 'temperature.gif')
geemap.download_ee_video(tempCol, videoArgs, saved_gif)
geemap.show_image(saved_gif)

In [ ]:
out_gif = os.path.join(out_path, 'temperature2.gif')
text = [str(n).zfill(2) + ":00" for n in range(0, 24)]
geemap.add_text_to_gif(saved_gif,
                       out_gif,
                       xy=('3%', '5%'),
                       text_sequence=text,
                       font_size=30,
                       font_color='#ffffff')
geemap.add_text_to_gif(out_gif,
                       out_gif,
                       xy=('32%', '92%'),
                       text_sequence='NOAA GFS Hourly Temperature',
                       font_color='white')
geemap.show_image(out_gif)

In [ ]:
# 下载一个GIF
url = 'https://i.imgur.com/MSde1om.gif'
out_path = r'E:/GEE'
geemap.download_from_url(url, out_file_name='temp.gif', out_dir=out_path)
in_gif = os.path.join(out_path, 'temp.gif')
geemap.show_image(in_gif)

In [ ]:
noaa_logo = 'https://bit.ly/3ahJoMq'
out_gif = os.path.join(out_path, 'output.gif')

# 为GIF添加logo
geemap.add_image_to_gif(in_gif,
                 out_gif,
                 in_image=noaa_logo,
                 xy=('2%', '80%'),
                 image_size=(80, 80))
geemap.show_image(out_gif)

In [ ]:
# 定义一个colorbar
width = 250
height = 30
palette = ['blue', 'purple', 'cyan', 'green', 'yellow', 'red']
labels = [-40, 35]
colorbar = geemap.create_colorbar(width=width,
                                  height=height,
                                  palette=palette,
                                  vertical=False,
                                  add_labels=True,
                                  font_size=20,
                                  labels=labels)
# 将定义的colorbar 加载到图像中，其本质还是添加图像
geemap.add_image_to_gif(out_gif,
                          out_gif,
                          in_image=colorbar,
                          xy=('69%', '89%'),
                          image_size=(250, 250))
geemap.show_image(out_gif)

# 4  数据分析

## 4.1  聚类

In [ ]:
# 导入特征文件
file_path = os.path.abspath('../data/us-cities.json')
# 如果没有该文件则请求下载此文件
if not os.path.exists(file_path):
    url = 'https://github.com/giswqs/geemap/raw/master/examples/data/us-cities.json'
    r = requests.get(url)
    with open(file_path, 'w') as f:
        f.write(r.content.decode("utf-8"))        

with open(file_path) as f:
    json_data = json.load(f)
    
json_data['features'][:4]

In [ ]:
Map = geemap.Map()

maker_cluster = MarkerCluster(
    markers=[Marker(location=feature['geometry']['coordinates'][::-1]) for feature in json_data['features']],
    name = 'Markers')

Map.add_layer(maker_cluster)
Map

## 4.2  区域分析

In [ ]:
# 加载DEM数据
dem = ee.Image('USGS/SRTMGL1_003')

dem_vis = {
  'min': 0,
  'max': 4000,
  'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']}

Map = geemap.Map()
Map.addLayer(dem, dem_vis, 'SRTM DEM')

states = ee.FeatureCollection("TIGER/2018/States")
Map.addLayer(states, {}, 'US States')

Map

In [ ]:
out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
out_dem_stats = os.path.join(out_dir, 'dem_stats.csv')  

# 可输出文件类型: csv, shp, json, kml, kmz
# 可选择区域计算指标: MEAN, MAXIMUM, MINIMUM, MEDIAN, STD, MIN_MAX, VARIANCE, SUM
geemap.zonal_statistics(dem, states, out_dem_stats, statistics_type='MEAN', scale=1000)

In [ ]:
landcover = ee.Image('USGS/NLCD/NLCD2016').select('landcover')
Map.addLayer(landcover, {}, 'NLCD 2016')
Map.add_legend(builtin_legend='NLCD')

out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
nlcd_stats = os.path.join(out_dir, 'nlcd_stats.csv')  

# statistics_type can be either 'SUM' or 'PERCENTAGE'
# denominator can be used to convert square meters to other areal units, such as square kilimeters
geemap.zonal_statistics_by_group(landcover, states, nlcd_stats, statistics_type='SUM', denominator=1000000, decimal_places=2)

# 5  实战练习

## 5.1  生成Landsat gif

目标：生成1985年以来Landsat图像的Gif动图

In [ ]:
# 解法1：
# 定义区域
roi = ee.Geometry.Polygon([[[-115.471773, 35.892718], [-115.471773, 36.409454],
                            [-114.271283, 36.409454], [-114.271283, 35.892718],
                            [-115.471773, 35.892718]]], None, False)

collection = geemap.landsat_timeseries(roi=roi,
                                       start_year=1985,
                                       end_year=2019,
                                       start_date='01-10',
                                       end_date='12-20')
print(collection.size().getInfo())

# Define arguments for animation function parameters.
video_args = {
    'dimensions': 768,
    'region': roi,
    'framesPerSecond': 10,
    'bands': ['NIR', 'Red', 'Green'],
    'min': 0,
    'max': 4000,
    'gamma': [1, 1, 1]
}

# 下载gif
out_gif = os.path.join(out_path, "landsat_ts.gif")
geemap.download_ee_video(collection, video_args, out_gif)

# 添加文本
landsat_gif = os.path.join(out_path, "landsat_ts_text.gif")
geemap.add_text_to_gif(out_gif,
                       landsat_gif,
                       xy=('3%', '5%'),
                       text_sequence=1985,
                       font_size=30,
                       font_color='#ffffff',
                       add_progress_bar=False)

# 添加进度条
label = 'Urban Growth in Las Vegas'
geemap.add_text_to_gif(landsat_gif,
                       landsat_gif,
                       xy=('2%', '88%'),
                       text_sequence=label,
                       font_size=30,
                       font_color='#ffffff',
                       progress_bar_color='cyan')
geemap.show_image(landsat_gif)

In [ ]:
# 解法2
# 定义区域
roi = ee.Geometry.Polygon([[[-115.471773, 35.892718], [-115.471773, 36.409454],
                            [-114.271283, 36.409454], [-114.271283, 35.892718],
                            [-115.471773, 35.892718]]], None, False)
label = 'Urban Growth in Las Vegas'

Map = geemap.Map()
Map.add_landsat_ts_gif(label=label,
                       start_year=1985,
                       bands=['Red', 'Green', 'Blue'],
                       font_color='white',
                       frames_per_second=10,
                       progress_bar_color='blue')
Map

## 5.2 批量练习

In [ ]:
geemap.ee_search()

In [ ]:
geemap.file_browser("I:/GEE教程/geemap/examples")

In [ ]:
out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
out_csv = os.path.join(out_dir, 'ee_api.csv')
geemap.ee_api_to_csv(out_csv)